In [0]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
os.chdir('/content/drive/My Drive/scatnetgpu')
!pwd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/scatnetgpu


In [0]:
!pip install Pillow
!pip install scipy==1.1.0
import pandas as pd
from pandas import ExcelWriter

from scipy.ndimage import imread

import skimage
import numpy as np
import pywt
import scipy.stats

import cv2

!pip install cpbd
import cpbd
from matplotlib import pyplot as plt
plt.ion()
import glob

from scipy import io 
import itertools
from scipy import interp

     |████████████████████████████████| 31.2MB 101kB/s 
ERROR: tensorflow 2.2.0rc3 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Created wheel for cpbd: filename=cpbd-1.0.7-cp36-none-any.whl size=7482 sha256=1c1ba4d549a03079ac537a3010f6152cfe7f5abe1f0d977b972a70aa52ec0281
  Stored in directory: /root/.cache/pip/wheels/18/52/69/ae4ae36fe97718f94543c72acd25b66b269b5b82a7e30e32cb
Successfully built cpbd


In [0]:
import skimage.feature
def effective_mean(patch):
    vec=patch.flatten()
    m1 = [i for i in vec if i!=0]
    return(np.mean(m1))

def rgb2gray(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray


def ReturnPixelStats(img):
    f = []
    #Pixel Statistics
    for channel in range(0,3):
        a = img[:,:,channel]
        vec = a.flatten()
        # 8 statistic characteristics of pixel values for each color
        f.append(np.mean(vec))
        f.append(np.var(vec))
        f.append(scipy.stats.skew(vec))
        f.append(scipy.stats.kurtosis(vec))
        f.append(np.percentile(vec,25))
        f.append(np.median(vec))
        f.append(np.percentile(vec,75))
        f.append(scipy.stats.entropy(vec))
    return f


def ReturnPixelStatsGrey(img):

    f = []
    vec = img
    #Pixel Statistics
    # 8 statistic characteristics of pixel values for each color
    f.append(np.mean(vec))
    f.append(np.var(vec))
    f.append(scipy.stats.skew(vec))
    f.append(scipy.stats.kurtosis(vec))
    f.append(np.percentile(vec,25))
    f.append(np.median(vec))
    f.append(np.percentile(vec,75))
    f.append(scipy.stats.entropy(vec))
    return f

def ExtractTexture(img):
    glcm = skimage.feature.greycomatrix(img,[1],[0, np.pi/4, np.pi/2, 3*np.pi/4],normed=True, symmetric=True)
    contrast = skimage.feature.greycoprops(glcm, prop = 'contrast')
    homogeneity = skimage.feature.greycoprops(glcm, prop = 'homogeneity')
    dissimilarity = skimage.feature.greycoprops(glcm, prop = 'dissimilarity')
    ASM = skimage.feature.greycoprops(glcm, prop = 'ASM')
    correlation = skimage.feature.greycoprops(glcm, prop = 'correlation')
    energy = skimage.feature.greycoprops(glcm, prop = 'energy')
    features = (np.array([contrast, homogeneity, dissimilarity, ASM, correlation, energy])).flatten()
    return features


def get_wavelet_features_gray(img):
    wp = pywt.WaveletPacket2D(data=img, wavelet='haar', mode='symmetric')
    feature=[]
    data=wp['h'].data
    vec=data.flatten()
    feature.append(ReturnPixelStatsGrey(vec))

    data=wp['v'].data
    vec=data.flatten()
    feature.append(ReturnPixelStatsGrey(vec))

    data=wp['d'].data
    vec=data.flatten()
    feature.append(ReturnPixelStatsGrey(vec))

    return feature




def ImageFeatures(img):
    (h,l) = img[:,:,0].shape
    #Initialize feature vector to 0
    f = []
    
    #Pixel Statistics
    f.extend(ReturnPixelStats(img))

    #Texture Features
    #Function Takes uint8 greyscale image as input
    grey_img = rgb2gray(img)
    a_8 = grey_img.astype(np.uint8)
    f.extend(ExtractTexture(a_8))
    
    #Extracting Region Specific Features
    a = grey_img
    A1 = a[:int(h/4),:int(l/4)]
    A2 = a[:int(h/4), int(5*l/24):int(5*l/24+l/4)]
    A3 = a[:int(h/4):,int(3*l/4):]
    A4 = a[int(5*h/24):int(5*h/24+h/4),:int(l/4)]
    A6 = a[int(5*h/24): int(5*h/24+h/4),int(l-l/4):]
    A7 = a[-int(h/4):,:int(l/4)]
    A8 = a[int(h-h/4):,int(5*l/24):int(5*l/24+l/4)]
    A9 = a[int(h-h/4):, int(l-l/4):]
    #Central Pixel Statistics
    A5_color = img[int(5*h/24):int(5*h/24+h/4), int(5*l/24):int(5*l/24+l/4),:]
    f.extend(ReturnPixelStats(A5_color))
    #features to define symmetricity of the fundus image
    m1=effective_mean(A1)
    m2=effective_mean(A2)
    m3=effective_mean(A3)
    m4=effective_mean(A4)
    m6=effective_mean(A6)
    m7=effective_mean(A7)
    m8=effective_mean(A8)
    m9=effective_mean(A9)
    d1 = m1-m9
    d2 = m2-m8
    d3 = m3-m7
    d4 = m4-m6
    f.append(d1)
    f.append(d2)
    f.append(d3)
    f.append(d4)
    
    #w=[]

    #wavelet features
    #w.extend(get_wavelet_features_gray(n[:,:,0]))
    #w.extend(get_wavelet_features_gray(n[:,:,1]))
    #w.extend(get_wavelet_features_gray(n[:,:,2]))
    #f.extend(w)

    
    
    

    #CPBD Blur Feature
    cp = cpbd.compute(grey_img)
    f.append(cp)
    return f


In [0]:
import cv2
import os
import pandas as pd

from skimage.color import rgb2hsv

import cv2
import numpy as np




import os
#os.chdir('/content/drive/My Drive/scatnetgpu/All_images')
os.chdir('/content/drive/My Drive/scatnetgpu/Onsite_all')
!pwd



#K = np.zeros((1200, 77),dtype=float)
K = np.zeros((400, 77),dtype=float)
cnt=0

for iter in range(1,101):
  filename = "{:03d}".format(iter) + '_' + str(1) +'.jpg'
  #filename = "{:03d}".format(iter) + '_l' + str(1) +'.jpg'
  isExist = os.path.exists(os.path.join("/content/drive/My Drive/scatnetgpu/Onsite_all/",filename)) 
  #isExist = os.path.exists(os.path.join("/content/drive/My Drive/scatnetgpu/All_images/",filename)) 
  
  if isExist:
    for i in range(1,5):
      if (i<3):
        filename = "{:03d}".format(iter) + '_' + str(i) +'.jpg'
        #filename = "{:03d}".format(iter) + '_l' + str(i) +'.jpg'
      else:
        #filename = "{:03d}".format(iter) + '_r' + str(i-2) +'.jpg'
        filename = "{:03d}".format(iter) + '_' + str(i-2) +'.jpg'
      
      Im = cv2.imread(os.path.join("/content/drive/My Drive/scatnetgpu/Onsite_all/",filename))
      #Im = cv2.imread(os.path.join("/content/drive/My Drive/scatnetgpu/All_images/",filename))

      # ----  polar coordinates  ------
      Im = Im.astype(np.float32)
      value = np.sqrt(((Im.shape[0]/2.0)**2.0)+((Im.shape[1]/2.0)**2.0))
      
      polar_image = cv2.linearPolar(Im,(Im.shape[0]/2, Im.shape[1]/2), value, cv2.WARP_FILL_OUTLIERS)
      polar_image = polar_image.astype(np.uint8)
      #from google.colab.patches import cv2_imshow

      img = polar_image
      
      (h,l) = img[:,:,0].shape
      #Initialize feature vector to 0
      f = []
      #Pixel Statistics
      f.extend(ReturnPixelStats(img))
      #Texture Features
      #Function Takes uint8 greyscale image as input
      grey_img = rgb2gray(img)
      a_8 = grey_img.astype(np.uint8)
      f.extend(ExtractTexture(a_8))

      #Extracting Region Specific Features
      a = grey_img
      A1 = a[:int(h/4),:int(l/4)]
      A2 = a[:int(h/4), int(5*l/24):int(5*l/24+l/4)]
      A3 = a[:int(h/4):,int(3*l/4):]
      A4 = a[int(5*h/24):int(5*h/24+h/4),:int(l/4)]
      A6 = a[int(5*h/24): int(5*h/24+h/4),int(l-l/4):]
      A7 = a[-int(h/4):,:int(l/4)]
      A8 = a[int(h-h/4):,int(5*l/24):int(5*l/24+l/4)]
      A9 = a[int(h-h/4):, int(l-l/4):]
      #Central Pixel Statistics
      A5_color = img[int(5*h/24):int(5*h/24+h/4), int(5*l/24):int(5*l/24+l/4),:]
      f.extend(ReturnPixelStats(A5_color))
      #features to define symmetricity of the fundus image
      m1=effective_mean(A1)
      m2=effective_mean(A2)
      m3=effective_mean(A3)
      m4=effective_mean(A4)
      m6=effective_mean(A6)
      m7=effective_mean(A7)
      m8=effective_mean(A8)
      m9=effective_mean(A9)
      d1 = m1-m9
      d2 = m2-m8
      d3 = m3-m7
      d4 = m4-m6
      f.append(d1)
      f.append(d2)
      f.append(d3)
      f.append(d4)
      #CPBD Blur Feature
      cp = cpbd.compute(grey_img)
      f.append(cp)

      
      K[cnt,:] = f
      cnt = cnt+1
      print(cnt)
      
      #if cnt in range(100,1200,200):
TradFeat = K
TradFeat = pd.DataFrame(TradFeat)
import pandas as pd
from pandas import ExcelWriter


wr = ExcelWriter('On_Trad_feats_apr_22_polar.xlsx')
TradFeat.to_excel(wr,'sheet1')
wr.save()



/content/drive/My Drive/scatnetgpu/Onsite_all


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [0]:
print(i)

NameError: ignored